In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from google.colab import drive

In [2]:
# Google Drive 마운트
drive.mount('/content/drive')

# 기본 경로 설정
base_path = '/content/drive/My Drive/Colab Notebooks/homerun/'
arrival_data_path = f'{base_path}arrival_data2.csv'
gstation_path = f'{base_path}gStation2.csv'

def time_to_minutes(time_str):
    """시간 문자열을 분으로 변환"""
    hours, minutes = map(int, time_str.split(':'))
    return hours * 60 + minutes

def minutes_to_time(minutes):
    """분을 시간 문자열로 변환"""
    hours = minutes // 60
    mins = minutes % 60
    return f"{hours:02d}:{mins:02d}"

# 1. 데이터 로드
print("데이터 로드 중...")
arrival_df = pd.read_csv(arrival_data_path)
gstation_df = pd.read_csv(gstation_path)

Mounted at /content/drive
데이터 로드 중...


In [3]:
# 2. arrival_data 전처리
print("데이터 전처리 중...")
arrival_df['DEPARTURE_MINUTES'] = arrival_df['ARRIVAL_TIME'].apply(time_to_minutes)
arrival_df['HOUR'] = arrival_df['DEPARTURE_MINUTES'] // 60
arrival_df['IS_PEAK'] = arrival_df['HOUR'].isin([8, 9, 17, 18]).astype(int)
arrival_df['IS_SCHOOL'] = (arrival_df['DEPART_AT'] == 'SCH').astype(int)

데이터 전처리 중...


In [4]:
# 3. 학습 데이터 준비
X = arrival_df[['HOUR', 'IS_PEAK', 'IS_SCHOOL']]
y = arrival_df['DEPARTURE_MINUTES']

In [5]:
# 4. 모델 학습
print("\n모델 학습 중...")
rf_model = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=42)
rf_model.fit(X, y)

def predict_departure_time(original_time, is_school, prev_time=None):
    """
    주어진 시간에 대한 새로운 출발 시간 예측

    Args:
        original_time: 원래 스케줄 시간
        is_school: 학교 출발 여부
        prev_time: 이전 출발 시간 (시간 순서 체크용)
    """
    minutes = time_to_minutes(original_time)
    hour = minutes // 60
    is_peak = 1 if hour in [8, 9, 17, 18] else 0

    features = [[hour, is_peak, is_school]]
    predicted_minutes = rf_model.predict(features)[0]

    # 원래 시간에서 ±15분 범위로 제한
    predicted_minutes = max(minutes - 15, min(minutes + 15, predicted_minutes))

    # 이전 시간이 있는 경우, 순서 제약 조건 적용
    if prev_time is not None:
        prev_minutes = time_to_minutes(prev_time)
        if is_school:  # 학교 출발인 경우, 이전 역 출발보다 빨라선 안 됨
            predicted_minutes = max(predicted_minutes, prev_minutes + 10)  # 최소 10분 차이
        else:  # 역 출발인 경우, 이전 학교 출발보다 늦어야 함
            predicted_minutes = max(predicted_minutes, prev_minutes + 10)  # 최소 10분 차이

    return minutes_to_time(int(predicted_minutes))


모델 학습 중...


In [6]:
# 5. 새로운 스케줄 생성
print("\n새로운 스케줄 생성 중...")
new_schedule = gstation_df.copy()
days = ['MON', 'TUE', 'WED', 'THU', 'FRI']

for day in days:
    school_col = f'{day}_SCHOOL_DEPART'
    station_col = f'{day}_STATION_DEPART'

    # 각 행을 순차적으로 처리
    for idx in range(len(new_schedule)):
        if idx == 0:
            # 첫 번째 행은 이전 시간 없이 처리
            school_time = predict_departure_time(
                new_schedule.at[idx, school_col],
                is_school=1
            )
            new_schedule.at[idx, school_col] = school_time

            # 학교 출발 후 역 도착은 최소 15분 후
            station_time = predict_departure_time(
                new_schedule.at[idx, station_col],
                is_school=0,
                prev_time=school_time
            )
            new_schedule.at[idx, station_col] = station_time
        else:
            # 이전 행의 역 출발 시간을 고려하여 학교 출발 시간 설정
            prev_station_time = new_schedule.at[idx-1, station_col]
            school_time = predict_departure_time(
                new_schedule.at[idx, school_col],
                is_school=1,
                prev_time=prev_station_time
            )
            new_schedule.at[idx, school_col] = school_time

            # 현재 학교 출발 시간을 고려하여 역 출발 시간 설정
            station_time = predict_departure_time(
                new_schedule.at[idx, station_col],
                is_school=0,
                prev_time=school_time
            )
            new_schedule.at[idx, station_col] = station_time



새로운 스케줄 생성 중...


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does

In [7]:
# 6. 새로운 스케줄 저장
output_path = f'{base_path}ml_schedule_v2.csv'
new_schedule.to_csv(output_path, index=False)
print(f"\n새로운 스케줄이 저장되었습니다: {output_path}")


새로운 스케줄이 저장되었습니다: /content/drive/My Drive/Colab Notebooks/homerun/ml_schedule_v2.csv


In [8]:
# 7. 스케줄 검증
print("\n스케줄 검증 결과:")
valid = True
for day in days:
    school_col = f'{day}_SCHOOL_DEPART'
    station_col = f'{day}_STATION_DEPART'

    for idx in range(len(new_schedule)):
        school_time = time_to_minutes(new_schedule.at[idx, school_col])
        station_time = time_to_minutes(new_schedule.at[idx, station_col])

        if station_time <= school_time:
            print(f"오류 발견 - {day} {idx+1}번째 행: 학교 출발({new_schedule.at[idx, school_col]}) -> "
                  f"역 출발({new_schedule.at[idx, station_col]})")
            valid = False

        if idx > 0:
            prev_station_time = time_to_minutes(new_schedule.at[idx-1, station_col])
            if school_time <= prev_station_time:
                print(f"오류 발견 - {day} {idx+1}번째 행: 이전 역 출발({new_schedule.at[idx-1, station_col]}) -> "
                      f"현재 학교 출발({new_schedule.at[idx, school_col]})")
                valid = False

if valid:
    print("모든 스케줄이 순서 제약 조건을 만족합니다.")
else:
    print("일부 스케줄에서 순서 제약 조건 위반이 발견되었습니다.")


스케줄 검증 결과:
모든 스케줄이 순서 제약 조건을 만족합니다.


In [9]:
# 8. 결과 샘플 출력
print("\n수정된 스케줄 샘플 (처음 5개):")
comparison = pd.DataFrame({
    '기존_학교출발': gstation_df['MON_SCHOOL_DEPART'],
    '새로운_학교출발': new_schedule['MON_SCHOOL_DEPART'],
    '기존_역출발': gstation_df['MON_STATION_DEPART'],
    '새로운_역출발': new_schedule['MON_STATION_DEPART']
})
print(comparison.head())


수정된 스케줄 샘플 (처음 5개):
  기존_학교출발 새로운_학교출발 기존_역출발 새로운_역출발
0    7:45    08:00   8:00   08:15
1    8:00    08:25   8:15   08:35
2    8:02    08:45   8:17   08:55
3    8:03    09:05   8:18   09:15
4    8:05    09:25   8:20   09:35
